##### Copyright 2020 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Devices

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/devices"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/devices.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/devices.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/devices.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq 

## 1. Validation basics

When you are looking to run an algorithm on a real quantum computer (not a simulated one), there are often many additional constraints placed on the circuits you would like to run. Qubit connectivity, algorithm layout and types of gates used in the circuit all become much more important. Cirq uses the abtract class `Device` to represent constraints of an actual quantum processor, an example implementation of a device can be seen in the `cirq_google.Sycamore` class:

In [ ]:
import cirq_google
my_device = cirq_google.Sycamore
my_device

All devices are capable of validing moments, operations and circuits with the `validate_***` method to verify if they would work on that device or not. You can check if the following operations work on the Sycamore device:

In [ ]:
q1, q2, q3 = cirq.GridQubit(7, 4), cirq.GridQubit(7, 5), cirq.GridQubit(7, 6)
op1 = cirq.X(cirq.GridQubit(7 ,7))

try:
  my_device.validate_operation(op1)
except Exception as e:
  print(e)

Above you used a qubit that wasn't on the device and was invalid. Most validate operations also take into account things like supported gates and connectivity as well:

In [ ]:
op1 = cirq.H(q1)
op2 = cirq_google.SYC(q1, q3)

try:
  my_device.validate_operation(op1)
except Exception as e:
  print(e)

try:
  my_device.validate_operation(op2)
except Exception as e:
  print(e)

These validation operations can also be used with full circuits:

In [ ]:
my_circuit = cirq.Circuit(
    cirq.PhasedXPowGate(phase_exponent=0.3)(q1),
    cirq.PhasedXPowGate(phase_exponent=0.3)(q2),
    cirq_google.SYC(q1, q2),
    cirq_google.SYC(q2, q3),
)
my_device.validate_circuit(my_circuit)

`my_circuit` satisfies all the device constraints and could be run on a Sycamore device.

## 2. Metadata features

Some devices will also expose additional information via the `metadata` property. You can look at the metadata information of the Sycamore device with:

In [ ]:
metadata = my_device.metadata
metadata

The Sycamore device is a 2d grid device that exposes a `cirq.GridDeviceMetadata` with a uniform set of gates across all the qubits as well as a connectivity graph. You can explore the properties below:

In [ ]:
type(metadata)

In [ ]:
qubit_set = metadata.qubit_set
nx_graph = metadata.nx_graph
gateset = metadata.gateset

print(gateset)

These metadata features can be useful when designing/building algorithms around certain device information in order to tailor them for that device.

## 3. The `cirq.Device` interface

For advanced users (such as vendors) it is also possible to implement your own Device with its own unique constraints and metadata information. Below you can implement our own fictituous device:

In [ ]:
class MyDevice(cirq.Device):
  """Five qubits on a line, supporting X/Y/Z and CZ between neighbors."""
  def __init__(self):
      self._qubits = set(cirq.LineQubit.range(5))
      self._supported_gates = cirq.Gateset(
          cirq.XPowGate, cirq.YPowGate, cirq.ZPowGate, cirq.CZPowGate
      )

  def validate_operation(self, operation):
      if any(x not in self._qubits for x in operation.qubits):
        raise ValueError("Using qubits not found on device.")

      if len(operation.qubits) == 2:
          p, q = operation.qubits
          if not p.is_adjacent(q):
            raise ValueError('Non-local interaction: {}'.format(repr(operation)))

      if operation not in self._supported_gates:
        raise ValueError("Unsupported operation type.")

and use it to validate circuits:

In [ ]:
my_device = MyDevice()

my_circuit = cirq.Circuit(
    cirq.X(cirq.LineQubit(0)),
    cirq.X(cirq.LineQubit(2)),
    cirq.X(cirq.LineQubit(4)),
    cirq.CZ(*cirq.LineQubit.range(2))
)

my_device.validate_circuit(my_circuit)

Success! You have used Devices in Cirq to validate circuits for compatability when running as well as explored metadata information on the device and implemented your own boilerplate device.